# Tarea 4


In [7]:
Pkg.add("Roots")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Roots
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [8]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of DataFrames...
INFO: Updating cache of Rmath...
INFO: Updating cache of DataFrames...
INFO: Computing changes...
INFO: Upgrading Compat: v0.9.2 => v0.9.3
INFO: Upgrading Rmath: v0.1.3 => v0.1.4
INFO: Building Rmath


In [9]:
using PyPlot, Roots, Plots

INFO: Recompiling stale cache file /home/pablorv/.julia/lib/v0.5/URIParser.ji for module URIParser.
This may mean module Compat does not support precompilation but is imported by a module that does.
ERROR: LoadError: Declaring __precompile__(false) is not allowed in files that are being precompiled.
 in require(::Symbol) at ./loading.jl:385
 in include_from_node1(::String) at ./loading.jl:488
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in process_options(::Base.JLOptions) at ./client.jl:239
 in _start() at ./client.jl:318
while loading /home/pablorv/.julia/v0.5/URIParser/src/URIParser.jl, in expression starting on line 10


LoadError: LoadError: Failed to precompile URIParser to /home/pablorv/.julia/lib/v0.5/URIParser.ji.
while loading In[9], in expression starting on line 1

In [10]:
include("AutomDiff_V2.jl") #Archivo donado por Daniel Martínez

AD

In [11]:
import AD: Dual, xdual


## Ejercicio 0: Velocidad de convergencia. \\


El objetivo de este ejercicio es relacionar, la velocidad de convergencia con que un punto fijo (o una órbita periódica, en el caso de los dos últimos incisos) atraen a puntos suficientemente cercanos, con la derivada del mapeo en el punto fijo (o ciclo periódico). La idea es, entonces, calcular primero el punto fijo y, después, medir cómo la distancia de los iterados sucesivos (de una condición inicial $x_0$) al punto fijo se comporta en el tiempo, para los siguientes mapeos:
$F(x) = x^2+0.25$ \\
$F(x) = 3x(1-x)$ \\
$F(x) = \exp(x-1)$ \\
$F(x) = x^2 - 1.25$ \\
$F(x) = \exp(x+1)$ \\

Deberán resolver algunas cosas intermedias. Por ejemplo, ¿qué tanto deben acercarse al punto fijo, a fin de evitar ruido numérico? ¿Qué hay que hacer en el caso en que el punto sea neutral (ni atractivo ni repulsivo)?
En los dos últimos incisos, el interés es en los ciclos de periodo 2.


En este problema las funciones a utilizar son:

In [16]:
f1(x) = x^2 + 0.25
f2(x) = 3x*(1 - x)
f3(x) = exp(x - 1)
f4(x) = x^2 - 0.25
f5(x) = exp(1-x)
f1prim(x)=2x
f2prim(x)=3-6x
f3prim(x)=exp(x-1)
f4prim(x)=2x
f5prim(x)=-exp(1-x)

f5prim (generic function with 1 method)

Para buscar los puntos fijos, usemos una función que calcule los ceros de las funciones fi-x.

In [29]:
function quadratic{T<:Real}(a::T, b::T, c::T)
    disc = b^2 - 4*a*c #Calculamos el valor del discriminante
    @assert a != 0 "Tu ecuación no es de segundo grado, revisa los argumentos.", disc < 0 
    "La ecuación tiene soluciones complejas, no es posible resolver la ecuación."
    resuelve_cuad(a,b,c, disc)
end

function resuelve_cuad(a, b, c, disc) #El algoritmo para encontrar las raíces
    sq = sqrt(disc)
    x1 = (-b+sq)/(2a)
    x2 = (-b-sq)/(2a)
    if x1 == x2
        raices = Float64[x1]
    else
        raices = typeof(x1)[x1,x2]
    end
    return raices
end

resuelve_cuad (generic function with 1 method)

In [ ]:
function distancias(f,puntos,tolera)
    for 
    

In [33]:
quadratic(1.0,-1.0,0.25)

1-element Array{Float64,1}:
 0.5


## Ejercicio 1.
Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)$, donde la órbita de periodo $2^n$ nace. Es decir, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc.
A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde
$$
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
$$
La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.
Hint: Para realizar este ejercicio deben calcular el atractor para varias valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Sin embargo, van a requerir suficiente cuidado para obtener una buena aproximación de $c_n$.
Una opción, que tiene ciertos inconvenientes numéricos que también ciertas ventajas se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ usando los polinomios $Q_c^p(x)$ y diferenciación automática.

In [19]:
function puntosfijos(F::Function, interv)
    rf = fzeros(x -> (F(x) - x) , interv) #Método para encontrar los puntos fijos con la función fzeros de Roots.
    return rf
end


# Con esta función se construyen las Qc , al iterar y asignarles el nombre adecuado en cada iteración. 

Qc(x,c) = x^2 + c
nombre(n::Int, c) = Symbol( string("Qc", n, c) ) 

function Qcn(n::Int, c, y)
    
    mapeo = :(x -> x^2 + $c)
    for i = 2:n
        mapeo = :(x -> $(mapeo)(x^2 + $c))
    end
    mapback = :($(nombre(n, y))(x) = $mapeo(x))
    return mapback
end
#Lo que hicimos fue buscar cuándo el valor de la derivada es (-1)^(p+1) (con p el periodo de cada órbita) , 
#al variar la c para los puntos fijos de esa Qc, y luego evaluando la derivada.
#Se buscan alredeor de cada punto fijo dado por el orden del periodo 2^n , y se guardan en cn. 
function bifurca(p::Int, dominioc::FloatRange{Float64}, cn)
    fn = Function[]
    xd = xdual(0.0)
    count = 1
    for i in dominioc
        x0 = 0.0
        iterada = Qcn(p, i, count)
        push!(fn, eval(iterada))
        x0 = puntosfijos(fn[end], [-2, 2])
        if length(x0) > 0
            xd = xdual(x0[1])
            fxd = fn[end](xd)
            if abs(fxd.der - (1)) < 1e-5
                push!(cn, i)
            end
        end
        count += 1
    end
    return nothing
end

bifurca (generic function with 1 method)

Si probamos con un intervalo donde se encuentren varios puntos fijos tenemos que, para la cuarta iteración, obtenemos los c0,c1 y c2. 

In [17]:

bifurca(4,(-0.75-1/2^6):-1/2^6:-2, cn1)
cn1

4-element Array{Float64,1}:
  0.25
 -0.75
 -1.25
 -1.25

Pero ya no funciona para el C3, sospechamos que debemos encontrar otro método de obtener los puntos fijos. 